In [2]:
import pandas as pd

In [3]:
import chardet

# Detect encoding
with open('Data/train.csv', 'rb') as f:
    result = chardet.detect(f.read())

# Use detected encoding to read the file
dataset = pd.read_csv("Data/train.csv", encoding=result['encoding'])

In [4]:
dataset.head()

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [5]:
dataset=dataset.drop(["textID","selected_text","Time of Tweet","Age of User","Country","Population -2020","Land Area (Km²)","Density (P/Km²)"],axis=1)

In [6]:
dataset1=dataset[0:1000]

In [7]:
dataset=dataset1
dataset

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative
...,...,...
995,"I am twittering, LIKE A BOSS. Thanks Savvv",positive
996,my sleep pattern is screwed i need to try and...,positive
997,Since the demise of Woolworths it isn`t easy ...,negative
998,may the fourth be with you! happy star wars day,positive


In [8]:
#One-hot encoded
df_encoded = pd.get_dummies(dataset, columns=['sentiment'])

In [9]:
#making y
y=df_encoded[['sentiment_negative', 'sentiment_neutral', 'sentiment_positive']]

In [10]:
x=df_encoded.drop(['sentiment_negative','sentiment_neutral','sentiment_positive'],axis=1)

In [11]:
x

,text
0,"I`d have responded, if I were going"
1,Sooo SAD I will miss you here in San Diego!!!
2,my boss is bullying me...
3,what interview! leave me alone
4,"Sons of ****, why couldn`t they put them on t..."
...,...
995,"I am twittering, LIKE A BOSS. Thanks Savvv"
996,my sleep pattern is screwed i need to try and...
997,Since the demise of Woolworths it isn`t easy ...
998,may the fourth be with you! happy star wars day


In [12]:
import re
import nltk
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))


corpus=[]
stemmer = PorterStemmer()
for i in range(0, x.shape[0]):
    review = re.sub('[^a-zA-Z]', ' ', str(x.iloc[i]['text']))
    review = review.lower().split()
    review = [stemmer.stem(word) for word in review if not word in STOPWORDS]
    review = ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ashut\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
corpus

['respond go',
 'sooo sad miss san diego',
 'boss bulli',
 'interview leav alon',
 'son put releas alreadi bought',
 'http www dothebounci com smf shameless plug best ranger forum earth',
 'feed babi fun smile coo',
 'soooo high',
 '',
 'journey wow u becam cooler hehe possibl',
 'much love hope reckon chanc minim p never gonna get cake stuff',
 'realli realli like song love stori taylor swift',
 'sharpi run danger low ink',
 'want go music tonight lost voic',
 'test test lg env',
 'uh oh sunburn',
 'ok tri plot altern speak sigh',
 'sick past day thu hair look wierd didnt hat would look http tinyurl com mnf kw',
 'back home gonna miss everi one',
 'he',
 'oh marli sorri hope find soon',
 'play ghost onlin realli interest new updat kirin pet metamorph third job wait dragon pet',
 'clean hous famili com later today',
 'gotta restart comput thought win suppos put end constant rebooti',
 'see wat mean bout foll w friiday call lose f llower friday smh',
 'free fillin app ipod fun im addict

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 2500)
x = cv.fit_transform(corpus).toarray()

In [15]:
y

,sentiment_negative,sentiment_neutral,sentiment_positive
0,0,1,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
995,0,0,1
996,0,0,1
997,1,0,0
998,0,0,1


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [17]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train_scl = scaler.fit_transform(X_train)
X_test_scl = scaler.transform(X_test)

In [17]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier()
model_rf.fit(X_train_scl, y_train)

RandomForestClassifier()

In [18]:
print("Training Accuracy :", model_rf.score(X_train_scl, y_train))
print("Testing Accuracy :", model_rf.score(X_test_scl, y_test))

Training Accuracy : 0.9971407777084633
Testing Accuracy : 0.6357792601576713


In [19]:
#XGBoost
from xgboost import XGBClassifier
model_xgb = XGBClassifier()
model_xgb.fit(X_train_scl, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [20]:
print("Training Accuracy :", model_xgb.score(X_train_scl, y_train))
print("Testing Accuracy :", model_xgb.score(X_test_scl, y_test))

Training Accuracy : 0.6683302141817425
Testing Accuracy : 0.6343238326258338


In [21]:
#Decision tree
from sklearn.tree import DecisionTreeClassifier
model_dt = DecisionTreeClassifier()
model_dt.fit(X_train_scl, y_train)

DecisionTreeClassifier()

In [22]:
print("Training Accuracy :", model_dt.score(X_train_scl, y_train))
print("Testing Accuracy :", model_dt.score(X_test_scl, y_test))

Training Accuracy : 0.9963090039509254
Testing Accuracy : 0.6577319587628866


In [21]:
x_train_reshape=X_train_scl.reshape(X_train.shape[0],X_train.shape[1],1)
x_test_reshape=X_test_scl.reshape(X_test.shape[0],X_test.shape[1],1)

In [24]:
x_train_reshape.shape

(700, 2439, 1)

In [28]:
#RNN and LSTM
from keras.models import Sequential
from tensorflow.keras.layers import GRU,Embedding,SimpleRNN,LSTM,GRU,Dense, Flatten
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(2439, 1)))
model.add(LSTM(50,return_sequences=True))
model.add(Flatten())
model.add(Dense(3))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

C:\Users\ashut\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_8 (LSTM)                        │ (None, 2439, 50)            │          10,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ (None, 2439, 50)            │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 121950)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 3)                   │         365,853 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 396,453 (1.51 MB)

 Trainable params: 396,453 (1.51 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
model.fit(x_train_reshape, y_train,batch_size=256,epochs=10,validation_split=0.3)

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 113s 61s/step - accuracy: 0.2939 - loss: nan - val_accuracy: 0.3081 - val_loss: nan
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 122s 63s/step - accuracy: 0.3004 - loss: nan - val_accuracy: 0.3081 - val_loss: nan
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 130s 68s/step - accuracy: 0.2926 - loss: nan - val_accuracy: 0.3081 - val_loss: nan
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 133s 65s/step - accuracy: 0.2899 - loss: nan - val_accuracy: 0.3081 - val_loss: nan
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 139s 69s/step - accuracy: 0.3004 - loss: nan - val_accuracy: 0.3081 - val_loss: nan
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 126s 64s/step - accuracy: 0.2886 - loss: nan - val_accuracy: 0.3081 - val_loss: nan
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 126s 59s/step - accuracy: 0.2886 - loss: nan - val_accuracy: 0.3081 - val_loss: nan
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 134s 70s/step - accuracy: 0.2926 - loss: nan - val_accuracy: 0.3081 - val_loss: nan
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 123s

In [24]:
#Vader
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
review_list=dataset1['text'].astype(str).tolist()
sentiment_list=dataset1['sentiment'].tolist()
predictions=[]
for i in range(0,len(review_list),1):
    compound=sia.polarity_scores(review_list[i])['compound']
    if(compound>=0.05):
        predictions.append('positive')
    elif(compound<0.05 and compound>-0.05):
        predictions.append('neutral')
    elif(compound<=-0.05):
        predictions.append('negative')
matches=0
for i in range(len(predictions)):
    if(sentiment_list[i]==predictions[i]):
        matches=matches+1

print(f'Accuracy is {matches/len(predictions)}')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ashut\AppData\Roaming\nltk_data...


Accuracy is 0.6312725155562025


In [18]:
#RoBERTa
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from scipy.special import softmax
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
review_list=dataset1['text'].astype(str).tolist()
sentiment_list=dataset1['sentiment'].tolist()
predictions_rb=[]
for i in range(len(review_list)):
    try:
        encoding=tokenizer(review_list[i],return_tensors='tf')
        output = model(**encoding)
        scores = output.logits[0].numpy()
        scores = softmax(scores)
        scores_dict = {
          'neg' : scores[0],
          'neu' : scores[1],
          'pos' : scores[2]
        }
        if(scores_dict['neg']>scores_dict['neu'] and scores_dict['neg']>scores_dict['pos']):
            predictions_rb.append('negative')
        elif(scores_dict['pos']>scores_dict['neg'] and scores_dict['pos']>scores_dict['neu']):
            predictions_rb.append('positive')
        else:
            predictions_rb.append('neutral')
    except RuntimeError:
        print(f'Broke for row {i}')
        predictions_rb.append(predictions[i])
matches=0
for i in range(len(predictions_rb)):
    if(sentiment_list[i]==predictions_rb[i]):
        matches=matches+1
print(f'Accuracy is {matches/len(predictions_rb)}')


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


Accuracy is 0.725
